In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))


In [2]:
import ee 
import geemap

ee.Initialize(project="thurgau-irrigation")

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [3]:
from typing import Tuple
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from src.et_blue_per_field.et_blue_field_postprocessing import (
    compute_field_et_stats,
    compute_et_volume,
    threshold_et_volume,
)
from src.et_blue_per_field.etc_look_up_processing import (
    add_date_column,
    compute_et_ratio_timeseries,
    plot_multiple_et_ratio_timeseries,
    create_et_mask,
)

from utils.ee_utils import (
    back_to_float,
    back_to_int,
    export_image_to_asset,
    print_value_ranges,
    is_image_empty,
    fill_gaps_with_zeros,
    export_feature_collection,
)

from utils.date_utils import print_collection_dates, merge_same_date_images

---

## Constants

In [4]:
# YEAR = 2019

# ETC_THRESHOLD = 0.6
# MINIMUM_IRRIGATION_THRESHOLD = 0

# ET_PRODUCT = "Landsat_30m"


# PATH_TO_ET_BLUE_POSTPROCESSED = f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_postprocessed/ET_blue_postprocessed_{YEAR}_dekadal_from_WaPOR_10m"
# PATH_TO_FIELD_BOUNDARIES = (
#     f"projects/thurgau-irrigation/assets/FribourgAndVaud/blind_potato_fields"
# )
# PATH_TO_ET_GREEN = f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_{YEAR}_dekadal_from_WaPOR_10m"
# PATH_TO_ETC_LOOK_UP = (
#     f"projects/thurgau-irrigation/assets/FribourgAndVaud/ETc_WAPOR/ETc_Pasture_Broye"
# )

# TEMPORAL_RESOLUTION = "dekadal"
# SCALING_FACTOR = 100

# PATH_TO_POSTPROCESSED_FIELD_LEVEL_ET_BLUE = f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_per_field/ET_blue_per_field_{YEAR}_{TEMPORAL_RESOLUTION}_from_{ET_PRODUCT}_ETC_threshold_{int(ETC_THRESHOLD * 100)}"

## 0. Load the assets

In [5]:
# crop_fields = ee.FeatureCollection(PATH_TO_FIELD_BOUNDARIES)

In [6]:
# et_green_collection = (
#     ee.ImageCollection(PATH_TO_ET_GREEN)
#     .filterDate(f"{YEAR}-01-01", f"{YEAR}-12-31")
#     .map(lambda img: back_to_float(img, SCALING_FACTOR))
# )

# print_collection_dates(et_green_collection)

In [7]:
# ETc_look_up = ee.FeatureCollection(PATH_TO_ETC_LOOK_UP)

# ETc_look_up_df = geemap.ee_to_df(ETc_look_up)

# ETc_look_up_df = add_date_column(ETc_look_up_df)

# ETc_look_up_df["ETc"] = ETc_look_up_df["ETc"] / 10

# ETc_look_up_df = ETc_look_up_df.sort_values(by="Date")

# print(ETc_look_up_df.head(12))

In [8]:
# et_blue_postprocessed_collection = (
#     ee.ImageCollection(PATH_TO_ET_BLUE_POSTPROCESSED)
#     .filterDate(f"{YEAR}-01-01", f"{YEAR}-12-31")
#     .map(lambda img: back_to_float(img, SCALING_FACTOR))
# )

# print_collection_dates(et_blue_postprocessed_collection)

## 1. Mask ET blue pixels where ET green > X * ET<sub>c</sub>

In [9]:
# masked_collection = et_green_collection.map(lambda img: create_et_mask(img, ETc_look_up_df, "ET_green", ETC_THRESHOLD))

In [10]:
# def multiply_image_collections(
#     col1: ee.ImageCollection, col2: ee.ImageCollection, band_name1: str, band_name2: str
# ) -> ee.ImageCollection:
#     """
#     Multiply images from two collections with matching dates for a specified band.

#     Args:
#         col1: First image collection
#         col2: Second image collection
#         band_name1: Band name from first collection to multiply
#         band_name2: Band name from second collection to multiply

#     Returns:
#         Image collection containing the multiplied results
#     """
#     # Sort both collections by date
#     sorted_col1 = col1.sort("system:time_start")
#     sorted_col2 = col2.sort("system:time_start")

#     # Convert to lists for paired iteration
#     col_list1 = sorted_col1.toList(sorted_col1.size())
#     col_list2 = sorted_col2.toList(sorted_col2.size())

#     def multiply_images(index):
#         # Get corresponding images
#         img1 = ee.Image(col_list1.get(index))
#         img2 = ee.Image(col_list2.get(index))

#         # Multiply the specified bands
#         result = img1.select(band_name1).multiply(img2.select(band_name2))

#         # Return result with the timestamp from first collection
#         return result.copyProperties(img1, ["system:time_start"])

#     # Create sequence for mapping
#     sequence = ee.List.sequence(0, sorted_col1.size().subtract(1))

#     # Map multiplication over the sequences
#     return ee.ImageCollection(sequence.map(multiply_images))

In [11]:
# et_blue_masked = multiply_image_collections(et_blue_postprocessed_collection, masked_collection, "ET_blue", "ET_green")
# et_blue_masked_list = et_blue_masked.toList(et_blue_masked.size())

## 2. Compute ET blue statistics per field

In [12]:
# def count_empty_images_per_month(feature):
#     properties = feature.propertyNames().filter(
#         ee.Filter.stringContains("item", "median_et_blue")
#     )

#     def count_empty_by_month(current_month, prev_feature):
#         month_str = ee.Number(current_month).format("%02d")
#         month_props = properties.filter(ee.Filter.stringContains("item", month_str))

#         empty_count = month_props.map(
#             lambda prop: ee.Number(
#                 ee.Algorithms.If(ee.Number(feature.get(prop)).eq(-99), 1, 0)
#             )
#         ).reduce(ee.Reducer.sum())

#         prop_name = ee.String("empty_images_month_").cat(month_str)
#         return ee.Feature(prev_feature).set(prop_name, empty_count)

#     months = ee.List.sequence(4, 10)
#     return months.iterate(count_empty_by_month, feature)


In [13]:
# dates = et_blue_masked.aggregate_array("system:time_start")

# dates = [ee.Date(date).format("YYYY-MM-dd").getInfo() for date in dates.getInfo()]


# crop_fields_iteration = crop_fields

# crop_fields_iteration = crop_fields_iteration.map(
#     lambda f: f.set(
#         {
#             "etc_threshold": ETC_THRESHOLD,
#         }
#     )
# )

# for i, date in enumerate(dates):

#     image = ee.Image(et_blue_masked_list.get(i))

#     fileds_with_stats = compute_field_et_stats(
#         et_image=image,
#         fields=crop_fields_iteration,
#         et_band_name="ET_blue",
#         scale=10,
#         date=date,
#     )

#     feature_with_m3 = compute_et_volume(fileds_with_stats, date=date)

#     feature_with_m3 = threshold_et_volume(
#         feature_with_m3, threshold=MINIMUM_IRRIGATION_THRESHOLD, date=date
#     )

#     crop_fields_iteration = feature_with_m3


# updated_features = crop_fields_iteration.map(count_empty_images_per_month)

# task_name = f"field_et_blue_{ET_PRODUCT}_ETC_{ETC_THRESHOLD}_{YEAR}"

# export_feature_collection(
#     collection=updated_features,
#     task_name=task_name,
#     asset_id=PATH_TO_POSTPROCESSED_FIELD_LEVEL_ET_BLUE,
# )

# updated_features.getInfo()

---

In [14]:
# Map = geemap.Map()

# et_blue_list = et_blue_masked.toList(et_blue_masked.size())
# et_blue = ee.Image(et_blue_list.get(13))

# vis_params = {"bands": ["ET_blue"], "min": 0, "max": 1, "palette": "viridis"}

# Map.addLayer(et_blue, vis_params, "ET_blue")
# Map.addLayer(crop_fields_iteration, {"color": "red"}, "fields_with_stats")

# Map.centerObject(crop_fields, 12)


# Map

---

In [15]:
# etc_table = ee.FeatureCollection("projects/thurgau-irrigation/assets/FribourgAndVaud/ETc_WAPOR/ETc_Pasture_Broye")

# etc_df = geemap.ee_to_df(etc_table)

# etc_df = add_date_column(etc_df)

# etc_df["ETc"] = etc_df["ETc"]/10

# print(etc_df.tail(12))


In [16]:
# et_green_collection1 = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_2020_dekadal_from_Landsat_30m"
# ).map(lambda img: back_to_float(img, SCALING_FACTOR))

# et_green_collection2 = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_2020_dekadal_from_WaPOR_10m"
# ).map(lambda img: back_to_float(img, SCALING_FACTOR))

In [17]:
# ratio_df = compute_et_ratio_timeseries(
#     et_collections=[et_green_collection1 ,et_green_collection2],
#     etc_df=etc_df,
#     et_band_name="ET_green"
# )

# # Plot the results
# plot_multiple_et_ratio_timeseries(ratio_df)

# # You can also examine the raw data
# print(ratio_df.head(36))

## Sanity check

In [18]:
YEAR = 2021
ET_PRODUCT = "WaPOR_10m"


PATH_TO_FEATURE_COLLECTION = f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_per_field/ET_blue_per_field_{YEAR}_dekadal_from_{ET_PRODUCT}_ETC_threshold_70"

fc = ee.FeatureCollection(PATH_TO_FEATURE_COLLECTION)

In [19]:
def get_days_in_month(month: str) -> int:
    """Return number of days in given month for {YEAR}."""
    days_lookup = {"04": 30, "05": 31, "06": 30, "07": 31, "08": 31, "09": 30, "10": 31}
    return days_lookup[month]


def calculate_monthly_sums(feature: ee.Feature) -> ee.Feature:
    months = ["04", "05", "06", "07", "08", "09", "10"]
    area_ha = feature.geometry().area().divide(10000)

    new_properties = {}
    for month in months:
        days_in_month = get_days_in_month(month)

        # Get dekadal values
        d01 = ee.Number.parse(feature.get(f"et_blue_m3_{YEAR}-{month}-01"))
        d11 = ee.Number.parse(feature.get(f"et_blue_m3_{YEAR}-{month}-11"))
        d21 = ee.Number.parse(feature.get(f"et_blue_m3_{YEAR}-{month}-21"))

        # Calculate days for each dekad
        if month in ["04", "06", "09"]:
            days = [10, 10, 10]  # 30-day month
        else:
            days = [10, 10, 11]  # 31-day month

        # Weight values by days
        weighted_sum = (
            d01.multiply(days[0]).add(d11.multiply(days[1])).add(d21.multiply(days[2]))
        )

        # Set properties
        sum_property = f"et_blue_m3_{YEAR}_{month}"
        new_properties[sum_property] = weighted_sum

        per_ha_property = f"et_blue_m3/ha_{YEAR}_{month}"
        new_properties[per_ha_property] = weighted_sum.divide(area_ha)

    return feature.set(new_properties)


def process_collection(fc: ee.FeatureCollection) -> ee.FeatureCollection:
    return fc.map(calculate_monthly_sums)

In [20]:
processed_fc = process_collection(fc)

processed_fc.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[6.840180777689016, 46.69885879703907],
    [6.840247639448058, 46.6987785210351],
    [6.840238700848878, 46.69873840208001],
    [6.840292219343092, 46.69863589523111],
    [6.8403636051081405, 46.69840398871507],
    [6.842303252946304, 46.69753000944118],
    [6.842784863222069, 46.69790014547235],
    [6.842147218449351, 46.69818099836846],
    [6.841670062779664, 46.69839951383048],
    [6.840354673223982, 46.698997080890166],
    [6.840180777689016, 46.69885879703907]]]},
 'id': '0000000000000000003a',
 'properties': {'ID': '20-1-510461',
  'Jahr': 20,
  'Kultur': 'Kartoffeln',
  'empty_images_month_04': 0,
  'empty_images_month_05': 0,
  'empty_images_month_06': 0,
  'empty_images_month_07': 0,
  'empty_images_month_08': 0,
  'empty_images_month_09': 0,
  'empty_images_month_10': 0,
  'et_blue_m3/ha_2021_04': 126.9999980926514,
  'et_blue_m3/ha_2021_05': 87.99999952316286,
  'et_blue_m3/ha_2021_06': 75,
  '

In [21]:
def filter_et_values(feature: ee.Feature, threshold: float) -> ee.Feature:
    months = ["04", "05", "06", "07", "08", "09", "10"]
    new_properties = {}

    threshold = ee.Number(threshold)

    feature = feature.set({"minimum_irrigation_threshold": threshold})

    for month in months:
        # Get number of empty images for the month
        empty_images = ee.Number(feature.get(f"empty_images_month_{month}"))

        # Calculate adjusted threshold
        adjusted_threshold = threshold.multiply(
            ee.Number(3).subtract(empty_images)
        ).divide(3)

        # Get and filter ET value
        property_name = f"et_blue_m3/ha_{YEAR}_{month}"
        value = ee.Number(feature.get(property_name))
        filtered_value = ee.Algorithms.If(value.lt(adjusted_threshold), 0, value)
        new_properties[f"filtered_{property_name}"] = filtered_value

    return feature.set(new_properties)


filtered_fc = processed_fc.map(lambda f: filter_et_values(f, 0))
for threshold in [100, 140]:
    filtered_fc = processed_fc.map(lambda f: filter_et_values(f, threshold))

    task_name = f"min_irr_{int(threshold)}_{YEAR}"

    export_name = PATH_TO_FEATURE_COLLECTION + f"_min_irr_{int(threshold)}"

    export_feature_collection(
        collection=filtered_fc,
        task_name=task_name,
        asset_id=export_name,
    )

    print(f"Exported {export_name} for {ET_PRODUCT}")


filtered_fc.first().getInfo()

Exported projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_per_field/ET_blue_per_field_2021_dekadal_from_WaPOR_10m_ETC_threshold_70_min_irr_100 for WaPOR_10m
Exported projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_per_field/ET_blue_per_field_2021_dekadal_from_WaPOR_10m_ETC_threshold_70_min_irr_140 for WaPOR_10m


{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[6.840180777689016, 46.69885879703907],
    [6.840247639448058, 46.6987785210351],
    [6.840238700848878, 46.69873840208001],
    [6.840292219343092, 46.69863589523111],
    [6.8403636051081405, 46.69840398871507],
    [6.842303252946304, 46.69753000944118],
    [6.842784863222069, 46.69790014547235],
    [6.842147218449351, 46.69818099836846],
    [6.841670062779664, 46.69839951383048],
    [6.840354673223982, 46.698997080890166],
    [6.840180777689016, 46.69885879703907]]]},
 'id': '0000000000000000003a',
 'properties': {'ID': '20-1-510461',
  'Jahr': 20,
  'Kultur': 'Kartoffeln',
  'empty_images_month_04': 0,
  'empty_images_month_05': 0,
  'empty_images_month_06': 0,
  'empty_images_month_07': 0,
  'empty_images_month_08': 0,
  'empty_images_month_09': 0,
  'empty_images_month_10': 0,
  'et_blue_m3/ha_2021_04': 126.9999980926514,
  'et_blue_m3/ha_2021_05': 87.99999952316286,
  'et_blue_m3/ha_2021_06': 75,
  '

In [22]:
# Map = geemap.Map(height="800px")


# irrigation_09 = processed_fc.reduceToImage(
#     properties=["et_blue_m3/ha_2018_08"], reducer=ee.Reducer.first()
# )

# Map.addLayer(
#     irrigation_09, {"min": 0, "max": 200, "palette": "viridis"}, "irrigation_08"
# )

# Map.add_colorbar({"min": 0, "max": 200, "palette": "viridis"})

# Map.addLayer(processed_fc, {}, "ET_blue_per_field")
# Map.centerObject(processed_fc, 12)

# Map

In [23]:
# import matplotlib.pyplot as plt
# import numpy as np
# from typing import List, Tuple
# import re

# def get_et_blue_properties(feature: ee.Feature) -> Tuple[List[float], float]:
#     """Extract positive dekadal et_blue values and area from feature."""
#     properties = feature.toDictionary().getInfo()
#     geometry = feature.geometry().area().getInfo()
#     area_ha = geometry / 10000  # Convert square meters to hectares
    
#     dekadal_pattern = re.compile(r'et_blue_m3_\d{4}-\d{2}-(01|11|21)$')
#     et_blue_values = [
#         float(value) for key, value in properties.items()
#         if dekadal_pattern.match(key) and 
#         isinstance(value, (int, float)) and 
#         value > 0
#     ]
#     return et_blue_values, area_ha

# def calculate_field_means(fc: ee.FeatureCollection) -> List[float]:
#     """Calculate mean positive et_blue values per hectare for each field."""
#     means = []
#     features = fc.getInfo()['features']
    
#     for feature in features:
#         values, area = get_et_blue_properties(ee.Feature(feature))
#         if values:
#             # Normalize by area
#             normalized_values = [v / area for v in values]
#             means.append(np.mean(normalized_values))
    
#     return means

# def plot_histogram(means: List[float], bins: int = 30) -> None:
#     """Create histogram of mean et_blue values."""
#     plt.figure(figsize=(10, 6))
#     plt.hist(means, bins=bins, edgecolor='black')
#     plt.xlabel('Mean Positive ET Blue (m³/ha)')
#     plt.ylabel('Frequency')
#     plt.title('Distribution of Mean Positive Dekadal ET Blue Values')
#     plt.grid(True, alpha=0.3)
#     sns.despine()
#     plt.show()

# means = calculate_field_means(filtered_fc)
# plot_histogram(means)

## Final step: preparing the table for Keiser

In [29]:
YEAR = 2018
ET_PRODUCT = "WaPOR_10m"
MIN_IRRIGATION_THRESHOLD = 100  # m³/ha
ETC_THRESHOLD = 70  # %


PATH_TO_FEATURE_COLLECTION = f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_per_field/ET_blue_per_field_{YEAR}_dekadal_from_{ET_PRODUCT}_ETC_threshold_{ETC_THRESHOLD}_min_irr_{MIN_IRRIGATION_THRESHOLD}"

fc = ee.FeatureCollection(PATH_TO_FEATURE_COLLECTION)

In [30]:
fc.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[6.898340787574285, 46.82566675554755],
    [6.898380968295865, 46.82564443232396],
    [6.8985682164940565, 46.8256488679297],
    [6.898871397517888, 46.82567117592232],
    [6.900851242980219, 46.828560704623655],
    [6.900837916119662, 46.828565147649975],
    [6.900360784718916, 46.82873013401383],
    [6.900307299635416, 46.82870782203597],
    [6.899303962420583, 46.82718283774913],
    [6.898345301063176, 46.825706872132194],
    [6.898340787574285, 46.82566675554755]]]},
 'id': '00000000000000000000',
 'properties': {'ID': '18-1-510429',
  'Jahr': 18,
  'Kultur': 'Kartoffeln',
  'empty_images_month_04': 0,
  'empty_images_month_05': 0,
  'empty_images_month_06': 0,
  'empty_images_month_07': 0,
  'empty_images_month_08': 0,
  'empty_images_month_09': 0,
  'empty_images_month_10': 0,
  'et_blue_m3/ha_2018_04': 18.00000071525574,
  'et_blue_m3/ha_2018_05': 0,
  'et_blue_m3/ha_2018_06': 43.00000071525574,
  

In [31]:
from typing import List, Dict


def get_collection_path(year: int, min_irr: int, etc: float) -> str:
    """Generate path for feature collection based on parameters."""
    return f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_per_field/ET_blue_per_field_{year}_dekadal_from_{ET_PRODUCT}_ETC_threshold_{int(etc*100)}_min_irr_{min_irr}"


def calculate_yearly_sum(feature: ee.Feature, year: int) -> ee.Feature:
    """Calculate yearly sum of filtered ET blue values."""
    months = ["04", "05", "06", "07", "08", "09", "10"]
    properties = [f"filtered_et_blue_m3/ha_{year}_{month}" for month in months]
    yearly_sum = ee.Number(0)

    for prop in properties:
        yearly_sum = yearly_sum.add(ee.Number(feature.get(prop)))

    return feature.set("yearly_sum", yearly_sum)


def process_collections(years: List[int]) -> pd.DataFrame:
    """Process collections for multiple years and compute irrigation confidence."""
    min_irr_thresholds = [100, 140]
    etc_thresholds = [0.6, 0.7]

    # Dictionary to store yearly sums by ID and year
    id_sums: Dict[str, Dict[int, int]] = {}

    for year in years:
        for min_irr in min_irr_thresholds:
            for etc in etc_thresholds:
                # Load collection
                path = get_collection_path(year, min_irr, etc)
                fc = ee.FeatureCollection(path)

                # Calculate yearly sums
                fc_with_sums = fc.map(lambda f: calculate_yearly_sum(f, year))

                # Get results
                results = fc_with_sums.select(["ID", "yearly_sum"]).getInfo()

                # Process results
                for feature in results["features"]:
                    id_val = feature["properties"]["ID"]
                    yearly_sum = feature["properties"]["yearly_sum"]

                    if id_val not in id_sums:
                        id_sums[id_val] = {y: 0 for y in years}

                    if yearly_sum > 0:
                        id_sums[id_val][year] += 1

    # Create DataFrame
    df_data = []
    for id_val, year_counts in id_sums.items():
        row_data = {"ID": id_val}
        for year in years:
            row_data[f"confidence_{year} [%]"] = (year_counts[year] / 4.0) * 100
        df_data.append(row_data)

    return pd.DataFrame(df_data)

In [32]:
years = [2018, 2019, 2020, 2021]
results = process_collections(years)

In [34]:
results.to_csv(f"/Users/cooper/Desktop/irrigation-mapper/data/irrigation_confidence_{ET_PRODUCT}.csv", index=False)